In [ ]:
#numerical approach
import numpy as np
import matplotlib.pyplot as plt
#function to delet absorbing state fromstate space
def del_abs(x1,t1):
    size=len(x1[:,0])
    for i in range(size):
        if x1[size-1-i,t1]== 0:
            x1=np.delete(x1,size-1-i,0)
        if x1[size-1-i,t1]== N:
            x1=np.delete(x1,size-1-i,0)
    return x1

#function to make state spapec
def state_make(T,N):
    x=np.zeros([N+1,1])
    for i in range(N+1):
        x[i]=int(i)
    for t in range(1,T+1):
        x=del_abs(x,t-1)
        y=np.zeros([len(x[:,0])*3,t+1])
        for j in range(len(x[:,0])):
            y[j*3+0,0:t]=x[j,:]
            y[j*3+0,-1]=x[j,-1]-1
            y[j*3+1,0:t]=x[j,:]
            y[j*3+1,-1]=x[j,-1]+0
            y[j*3+2,0:t]=x[j,:]
            y[j*3+2,-1]=x[j,-1]+1
        x=y
    return x
a,b,c,d,w,N,T=3,0,5,1,1,5,1 #defind parameters of model
fixproan=np.zeros([4,8])
fixtimean=np.zeros([4,8])
numb_ab=0
z=[]
state=state_make(T,N) #creat space state
size=len(state[:,0])  #number of states
P=np.zeros([size,size])  #transition_probability
for i in range(size):
    if state[i,-1]==0 or state[i,-1]==N: #Find absorbing states
        P[i,i]=1
        numb_ab+=1
        z.append(i)
        continue
    nc=state[i,0] #Number of cooperatpr at time t-τ
    n0=state[i,-1] #Number of cooperator at time t
    fi=1-w+w*((nc-1)*a+(N-nc)*b)/(N-1) #fitness of mutants
    gi=1-w+w*((nc)*c+(N-nc-1)*d)/(N-1) #fitness of residents
    #find transition probability
    for j in range(size):
        if np.sum(np.abs(state[i,1:]-state[j,:-1]))==0:
            if state[i,-1]-state[j,-1]==-1:
                P[i,j]=((n0*fi)/(n0*fi+(N-n0)*gi))*((N-n0)/(N-1))
            if state[i,-1]-state[j,-1]==1:
                P[i,j]=(((N-n0)*gi)/(n0*fi+(N-n0)*gi))*((n0)/(N-1))
            if state[i,-1]-state[j,-1]==0:
                jj=j
    P[i,jj]=1-np.sum(P[i,:])
#find fundamental matrix
Q=P
R=np.zeros([size,numb_ab])
absorb_states=np.zeros([numb_ab,len(state[0,:])])
j=0
for i in range(size-1,-1,-1):
    if state[i,-1]==0 or state[i,-1]==N:
        Q=np.delete(Q,i,0)
        Q=np.delete(Q,i,1)
        R[:,numb_ab-1-j]=P[:,i]
        absorb_states[numb_ab-1-j,:]=state[i,:]
        j+=1     
for i in z:
    R=np.delete(R,size-1-i,0)
I=np.identity(size-numb_ab)
NN=np.linalg.inv(I-Q)
B=np.dot(NN,R)
#calculate fixation probability and time
fixp=0
for i in range(len(absorb_states[:,0])):
    if absorb_states[i,-1]==N:
        fixp+=B[0,i]
fixar=np.zeros([len(B[:,0])])
for i in range(len(absorb_states[:,0])):
    if absorb_states[i,-1]==N:
        fixar+=B[:,i]
fixt=np.sum(NN[0,:]*(fixar/fixp))
np.savetxt("fixation_probability.txt",fixp)
np.savetxt("fixation_time.txt",fixt)